In [290]:
from pandas import DataFrame, Series
import pandas as pd
import numpy as np
from datetime import datetime
import math
from __future__ import print_function
from operator import itemgetter
from collections import defaultdict

In [28]:
sensorData = pd.read_table("SensorData.txt")
activityData = pd.read_table("activitiesData.txt")

On élimine la première ligne de chaque jeux de données, ce qui servent à rien

In [29]:
sensorData = sensorData.ix[1:]
activityData = activityData.ix[1:]

In [30]:
activityData.head()

,Start time,End time,ID
1,25-Feb-2008 00:22:46,25-Feb-2008 09:34:12,10
2,25-Feb-2008 09:37:17,25-Feb-2008 09:38:02,4
3,25-Feb-2008 09:49:23,25-Feb-2008 09:53:28,13
4,25-Feb-2008 10:02:28,25-Feb-2008 10:12:42,5
5,25-Feb-2008 10:19:06,25-Feb-2008 16:55:38,1


On vérifie si il y a des valeurs manquantes dans notre jeux de données:

In [31]:
print sensorData.isnull().sum()

Start time    0
End time      1
ID            1
Val           1
dtype: int64


In [32]:
sensorData = sensorData[sensorData['End time'].notnull()]

In [33]:
sensorData.isnull().sum()

Start time    0
End time      0
ID            0
Val           0
dtype: int64

In [34]:
print len(sensorData)

1318


In [35]:
sensorData.iloc[-1]

Start time    23-Mar-2008 19:04:46
End time      23-Mar-2008 19:04:47
ID                              12
Val                              1
Name: 1318, dtype: object

In [36]:
print activityData.isnull().sum()

Start time    0
End time      1
ID            1
dtype: int64


In [37]:
activityData.iloc[-1]

Start time    Length: 245
End time              NaN
ID                    NaN
Name: 246, dtype: object

In [38]:
activityData = activityData[activityData['End time'].notnull()]

On crée les listes des activités et des capteurs:

In [39]:
activityLabel = Series(['leave house','use toilet','take shower','go to bed','prepare breakfast','prepare dinner',
                        'get drink'], index=[1,4,5,10,13,15,17])
activityLabel

1           leave house
4            use toilet
5           take shower
10            go to bed
13    prepare breakfast
15       prepare dinner
17            get drink
dtype: object

In [40]:
sensorLabel = Series(['Microwave','Hall-Toilet door','Hall-Bathroom door','Cups cupboard','Fridge',
                      'Plates cupboard','Front door','Dishwasher','ToiletFlush','Freezer','Pans Cupboard',
                      'Washing machine','Groceries Cupboard','Hall-Bedroom door'],
                     index = [1,5,6,7,8,9,12,13,14,17,18,20,23,24])
sensorLabel

1              Microwave
5       Hall-Toilet door
6     Hall-Bathroom door
7          Cups cupboard
8                 Fridge
9        Plates cupboard
12            Front door
13            Dishwasher
14           ToiletFlush
17               Freezer
18         Pans Cupboard
20       Washing machine
23    Groceries Cupboard
24     Hall-Bedroom door
dtype: object

Maintenant on cherche à gérer les 2 colonnes de temps : Start Time et End Time

In [41]:
x1 = '25-Feb-2008 00:22:46'
x2 = '25-Feb-2008 00:23:47'
y1 = pd.to_datetime(x1)
y2 = pd.to_datetime(x2)
print type(y2-y1)
print (y2-y1)/pd.Timedelta(1,'s')

<class 'pandas.tslib.Timedelta'>
61.0


In [42]:
activityData['Start time'] = pd.to_datetime(activityData['Start time'])

In [43]:
activityData['End time'] = pd.to_datetime(activityData['End time'])

In [44]:
sensorData['Start time'] = pd.to_datetime(sensorData['Start time'])

In [45]:
sensorData['End time'] = pd.to_datetime(sensorData['End time'])

Convert ID columns from string to int:

In [46]:
sensorData.ID = sensorData['ID'].apply(int)

In [47]:
activityData.ID = activityData['ID'].apply(int)

Convert to raw feature matrix:

In [48]:
def convert2RawFeatMatrix(trainingSensorData, trainingActivityData, timeStep):
    beginTime = 0 
    endTime = 0
    
    if sensorData['Start time'][1] < activityData['Start time'][1]: # we compare 1st time step of activity data vs sensor data
        beginTime = sensorData['Start time'][1]
    else:
        beginTime = activityData['Start time'][1]
        
    if sensorData['End time'][len(sensorData)] > activityData['End time'][len(activityData)]:
        endTime = sensorData['End time'][len(sensorData)]
    else:
        endTime = activityData['End time'][len(activityData)]
        
    duration = endTime - beginTime
    numberOfTimeStep = math.ceil(duration/pd.Timedelta(timeStep,'s'))
    
    #print beginTime
    
    featureMatrix = pd.DataFrame(0, index = sensorLabel.index, columns = np.arange(numberOfTimeStep))
    labels = Series(0, index = np.arange(numberOfTimeStep))
    
    for index,rowSensor in trainingSensorData.iterrows():
        differenceStart = rowSensor['Start time'] - beginTime
        differenceEnd = rowSensor['End time'] - beginTime
        
        startStep = int(differenceStart / pd.Timedelta(timeStep,'s'))
        endStep = int(differenceEnd / pd.Timedelta(timeStep,'s'))
        
        featureMatrix.loc[rowSensor.ID][startStep:endStep] = 1
        
    for index, rowActivity in trainingActivityData.iterrows(): 
        differenceStart = rowActivity['Start time'] - beginTime
        #print differenceStart
        differenceEnd = rowActivity['End time'] - beginTime
        
        startStep = int(differenceStart / pd.Timedelta(timeStep,'s'))
        #print startStep, 
        endStep = int(differenceEnd / pd.Timedelta(timeStep,'s'))
        #print endStep
        
        labels[startStep:endStep] = rowActivity.ID
        
    return featureMatrix, labels
    
        
                                                                              
    

In [49]:
featureMatrix, labels = convert2RawFeatMatrix(sensorData, activityData, 60)

In [53]:
featureMatrix.ix[:,565:584]


,565.0,566.0,567.0,568.0,569.0,570.0,571.0,572.0,573.0,574.0,575.0,576.0,577.0,578.0,579.0,580.0,581.0,582.0,583.0,584.0
1,0,0,0,0,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
8,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0
12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0
14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [54]:
labels[565:584]

565     0
566     0
567     0
568     0
569    13
570    13
571    13
572    13
573    13
574     0
575     0
576     0
577     0
578     0
579     0
580     0
581     0
582     5
583     5
584     5
dtype: int64

Convert to last sensor fired representation, in which the last sensor that changed state continues to give 1 and changes to 0 when a different sensor changes state.

In [282]:
def convert2LastFiredFeatureMatrix(trainingSensorData, trainingActivityData, timeStep):
    beginTime = 0 
    endTime = 0
    
    if sensorData['Start time'][1] < activityData['Start time'][1]: # we compare 1st time step of activity data vs sensor data
        beginTime = sensorData['Start time'][1]
    else:
        beginTime = activityData['Start time'][1]
        
    if sensorData['End time'][len(sensorData)] > activityData['End time'][len(activityData)]:
        endTime = sensorData['End time'][len(sensorData)]
    else:
        endTime = activityData['End time'][len(activityData)]
        
    duration = endTime - beginTime
    numberOfTimeStep = math.ceil(duration/pd.Timedelta(timeStep,'s'))
    
    #print beginTime
    
    featureMatrix = pd.DataFrame(0, index = sensorLabel.index, columns = np.arange(numberOfTimeStep))
    finalFeatureMatrix = pd.DataFrame(0, index = sensorLabel.index, columns = np.arange(numberOfTimeStep)) 
    labels = Series(0, index = np.arange(numberOfTimeStep))
    
    for index,rowSensor in trainingSensorData.iterrows():
        differenceStart = rowSensor['Start time'] - beginTime
        differenceEnd = rowSensor['End time'] - beginTime
        
        startStep = int(differenceStart / pd.Timedelta(timeStep,'s'))
        endStep = int(differenceEnd / pd.Timedelta(timeStep,'s'))
        
        featureMatrix.loc[rowSensor.ID][startStep] = 1
        featureMatrix.loc[rowSensor.ID][endStep] = 1
    
    x,y = np.where(featureMatrix == 1)
    lst = []
    for i in range(len(x)): 
        lst.append((x[i],y[i]))
        
    lstSorted = sorted(lst, key = lambda pos:pos[1])
    
    #print(lstSorted)
    
    for i in range(len(lstSorted)-1): 
        finalFeatureMatrix.iloc[lstSorted[i][0]][lstSorted[i][1]: lstSorted[i+1][1]-1] = 1 
        
    for index, rowActivity in trainingActivityData.iterrows(): 
        differenceStart = rowActivity['Start time'] - beginTime
        #print differenceStart
        differenceEnd = rowActivity['End time'] - beginTime
        
        startStep = int(differenceStart / pd.Timedelta(timeStep,'s'))
        #print startStep, 
        endStep = int(differenceEnd / pd.Timedelta(timeStep,'s'))
        #print endStep
        
        labels[startStep:endStep] = rowActivity.ID
        
    return finalFeatureMatrix, featureMatrix, labels
    
        
                                                                              
    

In [283]:
finalFeatureMatrix, featureMatrix, labels = convert2LastFiredFeatureMatrix(sensorData, activityData, 60)

In [284]:
finalFeatureMatrix.ix[:,39990:]

,39990.0,39991.0,39992.0,39993.0,39994.0,39995.0,39996.0,39997.0,39998.0,39999.0,40000.0,40001.0,40002.0,40003.0,40004.0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0
13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [285]:
featureMatrix.ix[:,39990:]

,39990.0,39991.0,39992.0,39993.0,39994.0,39995.0,39996.0,39997.0,39998.0,39999.0,40000.0,40001.0,40002.0,40003.0,40004.0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [286]:
test = pd.DataFrame(0, index = sensorLabel.index, columns = np.arange(5)) 
test.iloc[2][1:2] = 111
test

,0,1,2,3,4
1,0,0,0,0,0
5,0,0,0,0,0
6,0,111,0,0,0
7,0,0,0,0,0
8,0,0,0,0,0
9,0,0,0,0,0
12,0,0,0,0,0
13,0,0,0,0,0
14,0,0,0,0,0
17,0,0,0,0,0


In [305]:
labels

0         0
1         0
2        10
3        10
4        10
5        10
6        10
7        10
8        10
9        10
10       10
11       10
12       10
13       10
14       10
15       10
16       10
17       10
18       10
19       10
20       10
21       10
22       10
23       10
24       10
25       10
26       10
27       10
28       10
29       10
         ..
39975     1
39976     1
39977     1
39978     1
39979     1
39980     1
39981     1
39982     1
39983     1
39984     1
39985     1
39986     1
39987     1
39988     1
39989     1
39990     1
39991     1
39992     1
39993     1
39994     1
39995     1
39996     1
39997     1
39998     1
39999     1
40000     1
40001     1
40002     1
40003     1
40004     1
dtype: int64

We now calculate the state transition frequencies

In [298]:
def labelFrequencyCounting(labels):
    activityDict = defaultdict(lambda: defaultdict(int)) #create a defaultdict of defaultdict
    
    for x in range(len(labels)-2): #we stop at the nth-1 number
        activityDict[labels[x]][labels[x+1]] += 1 
    return activityDict
    

In [299]:
labelFrequencyCounting = labelFrequencyCounting(labels)

In [300]:
labelFrequencyCounting

defaultdict(<function __main__.<lambda>>,
            {0: defaultdict(int,
                         {0: 4625,
                          1: 31,
                          4: 69,
                          5: 17,
                          10: 15,
                          13: 11,
                          15: 7,
                          17: 14}),
             1: defaultdict(int, {0: 22, 1: 22559, 4: 9, 15: 2}),
             4: defaultdict(int,
                         {0: 62,
                          1: 3,
                          4: 178,
                          5: 6,
                          10: 33,
                          13: 6,
                          15: 3,
                          17: 1}),
             5: defaultdict(int, {0: 23, 5: 219}),
             10: defaultdict(int, {0: 12, 4: 34, 10: 11565, 13: 2}),
             13: defaultdict(int, {0: 20, 13: 70}),
             15: defaultdict(int, {0: 5, 4: 2, 15: 333, 17: 5}),
             17: defaultdict(int, {0: 19, 13: 1, 17:

In [304]:
s = 0
for value in labelFrequencyCounting.values(): 
    s += sum(value.values())
s
        

40003